# **Estequiometría**

Resolvedor para combustión completa, obtenido de [link text](https://chemicals.readthedocs.io/chemicals.combustion.html#heat-of-combustion-and-stiochiometry).


---



# *Importación de librerías*


En primer lugar, con !pip instalamos los paquetes si Colab no los toma. Luego, debemos importar la librería chemicals, y lo hacemos con un alias. La librería cirpy la importamos sin alias.

In [ ]:
!pip install chemicals
!pip install cirpy
import chemicals as ch
import cirpy
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# *Consulta de CAS*

Con esto podemos hacer consultas sobre el CAS. Sin embargo, da varios valores, algunos 'deprecados' (obsoletos).

In [ ]:
cirpy.resolve('H2O', 'cas')

NameError: ignored

# *Ingreso de datos*

Ahora, debemos listar todos los elementos que participan de la reacción. Sea que estén en el combustible (fuel) o en el aire.

De cada uno de ellos debemos dar el [identificador CAS](https://commonchemistry.cas.org/) y la cantidad de átomos de la molécula.

Otros links útiles son:

1.   [Springer](https://materials.springer.com/substanceprofile/docs/smsid_flvtyceefwhjkvfw)
2.   [PubChem](https://pubchem.ncbi.nlm.nih.gov/compound/Oxygen#section=CAS)
3.   [rsc](https://www.rsc.org/periodic-table/element/8/oxygen)
4.   [echa.europa](https://echa.europa.eu/substance-information/-/substanceinfo/100.029.051)


Aquí ingresamos las fracciones molares del aire y del combustible. Aunque ciertos elementos no estén presentes en el aire, debemos incluirlos en la composición molar, porque así lo pide el código.

In [ ]:
nitrogen = {'name': 'nitrogen',
            'CAS': '7727-37-9',
            'atoms': ch.simple_formula_parser('N2'),
            'MW': ch.molecular_weight(ch.simple_formula_parser('N2')),
            'y_air': 0.79,
            'y_fuel': 0.025,
            }

oxygen = {'name': 'oxygen',
          'CAS': '7782-44-7',
          'atoms': ch.simple_formula_parser('O2'),
          'MW': ch.molecular_weight(ch.simple_formula_parser('O2')),
          'y_air': 0.205,
          'y_fuel': 0.025,
          }

water = {'name': 'water',
         'CAS': '7732-18-5',
         'atoms': ch.simple_formula_parser('H2O'),
         'MW': ch.molecular_weight(ch.simple_formula_parser('H2O')),
         'y_air': 0.0045,
         'y_fuel': 0.0005,
         }

carbon_dioxide = {'name': 'carbon_dioxide',
                  'CAS': '124-38-9',
                  'atoms': ch.simple_formula_parser('CO2'),
                  'MW': ch.molecular_weight(ch.simple_formula_parser('CO2')),
                  'y_air': 0.0005,
                  'y_fuel': 0.0005,
                  }

methane = {'name': 'methane',
           'CAS': '74-82-8',
           'atoms': ch.simple_formula_parser('CH4'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('CH4')),
           'y_air': 0,
           'y_fuel': 0.85,
           }

ethane = {'name': 'ethane',
          'CAS': '74-84-0',
          'atoms': ch.simple_formula_parser('C2H6'),
          'MW': ch.molecular_weight(ch.simple_formula_parser('C2H6')),
          'y_air': 0,
          'y_fuel': 0.07,
          }

propane = {'name': 'propane',
           'CAS' : '74-98-6',
           'atoms': ch.simple_formula_parser('C3H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C3H8')),
           'y_air': 0,
           'y_fuel': 0.029,
           }

substances = [nitrogen, oxygen, water, carbon_dioxide, methane, ethane, propane]

names = [substances[i]['name'] for i in range(len(substances))]
MW = [substances[i]['MW'] for i in range(len(substances))]
CAS = [substances[i]['CAS'] for i in range(len(substances))]
atoms = [substances[i]['atoms'] for i in range(len(substances))]
zs_air = [substances[i]['y_air'] for i in range(len(substances))]
zs_fuel = [substances[i]['y_fuel'] for i in range(len(substances))]

Para concluir la carga de datos, debemos dar o bien variables intensivas, o bien variables extensivas. Elegimos el caudal de combustible y el exceso de aire utilizado.

In [ ]:
n_fuel = 1.0     #Caudal molar en mol/s. Podemos cambiar la base para ponerlo en kg.
O2_excess = 0.15  #Exceso de aire.

# *Resolvedor*

Ejecutamos la resolución.

In [ ]:
ans = ch.fuel_air_spec_solver(zs_air = zs_air,
                              zs_fuel = zs_fuel,
                              CASs = CAS,
                              atomss = atoms,
                              n_fuel = n_fuel,
                              O2_excess = O2_excess)

MW_air = np.dot(np.array(MW), np.array(zs_air))
MW_fuel = np.dot(np.array(MW), np.array(zs_fuel))
MW_gases = np.dot(np.array(MW), np.array(ans['zs_out']))

# *Resultados*

Las zs son las fracciones molares de los gases de combustión (en base húmeda). Usa el orden que establecimos más arriba.
```
zs_gases = [gases['nitrogen'],
            gases['oxygen'],
            gases['water'],
            gases['carbon_dioxide'],
            gases['methane'],
            gases['ethane'],
            gases['propane'],
           ]
```



In [ ]:
[round(i, 5) for i in ans['zs_out']] #Redondeamos las fracciones molares de los gases de combustión.

[0.72561, 0.02475, 0.16411, 0.08553, 0.0, 0.0, 0.0]

Obtenemos la fracción molar del oxígeno en los gases de combustión, tanto en base húmeda como en base seca.

In [ ]:
round(ans['frac_out_O2'],5), round(ans['frac_out_O2_dry'],5)

(0.02475, 0.02961)

Obtenemos el caudal molar de aire necesario para realizar la combustión completa del caudal molar de combustible ingresado.

In [ ]:
ans['n_air']

11.602439024390247

Obtenemos el caudal molar de los gases de combustión húmedos.

In [ ]:
[round(i, 5) for i in ans['ns_out']]

[9.19093, 0.3135, 2.07871, 1.0833, 0.0, 0.0, 0.0]

Obtenemos las masas molares del aire, del combustible y de los gases húmedos.

In [ ]:
round(MW_air,2), round(MW_fuel,2), round(MW_gases,2)

(28.79, 18.55, 27.84)